In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
cd /content/gdrive/My\ Drive/OpenVino

/content/gdrive/My Drive/OpenVino


In [ ]:
!pip install pixellib

     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for imantics: filename=imantics-0.1.12-cp36-none-any.whl size=16034 sha256=0f44d3d5c269cc4c2ae463fe578b6451794a9cc27e7be4825383cfeee59a846e
  Stored in directory: /root/.cache/pip/wheels/73/93/1c/9e2fc52eb74441941bc76cac441ddcc2c7ad67b18e1849e62a
Successfully built imantics


In [ ]:
#!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [ ]:
import pixellib
from pixellib.instance import instance_segmentation

In [ ]:
segment_video = instance_segmentation()
segment_video.load_model("mask_rcnn_coco.h5")

Instructions for updating:
Use fn_output_signature instead


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import copy
import numpy as np


In [ ]:
def intersection_over_union(box_1, box_2):
    width_of_overlap_area = min(box_1['xmax'], box_2['xmax']) - max(box_1['xmin'], box_2['xmin'])
    height_of_overlap_area = min(box_1['ymax'], box_2['ymax']) - max(box_1['ymin'], box_2['ymin'])
    if width_of_overlap_area < 0 or height_of_overlap_area < 0:
        area_of_overlap = 0
    else:
        area_of_overlap = width_of_overlap_area * height_of_overlap_area
    box_1_area = (box_1['ymax'] - box_1['ymin']) * (box_1['xmax'] - box_1['xmin'])
    box_2_area = (box_2['ymax'] - box_2['ymin']) * (box_2['xmax'] - box_2['xmin'])
    area_of_union = box_1_area + box_2_area - area_of_overlap
    if area_of_union == 0:
        return 0
    return area_of_overlap / area_of_union

In [ ]:
cap = cv2.VideoCapture('traffic.mp4')
c=0

frames = []
boxes = {}

temp_queue = list(range(1,50))

frame_count_to_be_permanent = 5
count_to_vanish = 10

temp_ids = {}
permanent_ids = {}
previous_temp_ids = {}

current = -1
previous = -1

fourcc = cv2.VideoWriter_fourcc(*'XVID')


while (cap.isOpened()):

  ret, frame = cap.read() #read video frame
  
  frames+=[frame.copy()] #copy frame for further use
  
  if c==0:
    height, width, _ = frame.shape
    out = cv2.VideoWriter('output.avi', fourcc, 15.0, (width, height))




  segmask, output = segment_video.segmentFrame(frame, show_bboxes= True) #object detection
  
  boxes["frame_"+str(c)] = {'rois':[]}

  for rois, classes in zip(segmask['rois'], segmask['class_ids']):
    if classes == 3:
      boxes["frame_"+str(c)]['rois'] += [rois] #save rois

  print("Frame : "+ str(c))
  
  if len(temp_ids)==0:
    current = c

    for box in boxes["frame_"+str(c)]['rois']:
      temp_ids[str(temp_queue[0])] = {'box': box, 'frame_count': 0, 'same_count': 0}
      temp_queue.remove(temp_queue[0])

  else:
    previous_temp_ids = copy.deepcopy(temp_ids)
    previous = current
    current = c
    for box in boxes["frame_"+str(current)]['rois']:
      box_1 = {'ymin': box[0],'xmin': box[1],'ymax': box[2],'xmax': box[3]}

      max_iou = 0
      max_iou_index = -1
      for ids in temp_ids:
        previous_box = temp_ids[ids]['box']
        box_2 = {'ymin': previous_box[0],'xmin': previous_box[1],'ymax': previous_box[2],'xmax': previous_box[3]}
        t_iou = intersection_over_union(box_1, box_2)
        if t_iou > max_iou:
          max_iou = t_iou
          max_iou_index = ids
      
      #print(max_iou, max_iou_index)
      
      if max_iou>0.6:
        temp_ids[max_iou_index]['frame_count'] += 1
        temp_ids[max_iou_index]['box'] = box

      else:
        temp_ids[str(temp_queue[0])] = {'box': box, 'frame_count': 0, 'same_count': 0}
        temp_queue.remove(temp_queue[0])


    pop_ids = []
    for ids in temp_ids:
      if ids in previous_temp_ids:
        if temp_ids[ids]['frame_count']==previous_temp_ids[ids]['frame_count']:
          temp_ids[ids]['same_count'] += 1
        else:
          temp_ids[ids]['same_count'] = 0
      

      if temp_ids[ids]['frame_count']==frame_count_to_be_permanent and ids not in permanent_ids:
        permanent_ids[ids] = {}
        permanent_ids[ids]['vanish_count'] = 0


      if temp_ids[ids]['same_count']==count_to_vanish:
        #remove temp_ids
        pop_ids += [ids]

    for ids in pop_ids:
      if ids in permanent_ids:
        permanent_ids.pop(ids)
      temp_queue += [ids]
      xz = temp_ids.pop(ids)



#image visualize
  img = frames[c]
  for ids in temp_ids:
    box_show = temp_ids[ids]['box']
    cv2.rectangle(img, (box_show[1], box_show[0]), (box_show[3], box_show[2]), (255,0,0), 2)
    if ids in permanent_ids:
      cv2.putText(img, ids, (box_show[1], box_show[2]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    else:
      cv2.putText(img, ids, (box_show[1], box_show[2]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
    out.write(img)

  
  c+=1
  if c>50:
    break

cap.release()
out.release()

Frame : 0
Frame : 1
Frame : 2
Frame : 3
Frame : 4
Frame : 5
Frame : 6
Frame : 7
Frame : 8
Frame : 9
Frame : 10
Frame : 11
Frame : 12
Frame : 13
Frame : 14
Frame : 15
Frame : 16
Frame : 17
Frame : 18
Frame : 19
Frame : 20
Frame : 21
Frame : 22
Frame : 23
Frame : 24
Frame : 25
Frame : 26
Frame : 27
Frame : 28
Frame : 29
Frame : 30
Frame : 31
Frame : 32
Frame : 33
Frame : 34
Frame : 35
Frame : 36
Frame : 37
Frame : 38
Frame : 39
Frame : 40
Frame : 41
Frame : 42
Frame : 43
Frame : 44
Frame : 45
Frame : 46
Frame : 47
Frame : 48
Frame : 49
Frame : 50


In [ ]:
### boxes[ymin, xmin, ymax, xmax]